# Construct of general ANM matrix using PP
using PP for ANM is quite a lot more complicated and tedious than all-electron systems.
This note book attempt to create a readable/reusable script to construct PP ANM matrix

In [3]:
import qctoolkit as qtk
import numpy as np

In [4]:
# reuse code from PRM paper
cyl_map = [
        ['si', 5.431, [14, 14]],
        ['ge', 5.658, [32, 32]],
        ['sige', 5.432, [14, 32]],
        ['sn', 6.4892, [50, 50]],
        ['snsi', 5.961, [14, 50]],
        ['gesn', 6.0758, [32, 50]],
        ['gaas', 5.6535, [31, 33]],
        ['alp', 5.4635, [13, 15]],
        ['alas', 5.660, [13, 33]],
        ['alsb', 6.1355, [13, 51]],
        ['gap', 5.451, [31, 15]],
        ['gasb', 6.09, [31, 51]],
        ['inp', 5.86, [49, 15]],
        ['inas', 6.05, [49, 33]],
        ['insb', 6.47, [49, 51]],

]
print len(cyl_map)

15


In [10]:
# only CPMD PP interface is ready at the moment
# only Gamma-point calculations are required
qmsetting = {
    'program': 'cpmd',
    'kmesh': [1,1,1],
    'link_dep': False,
}

In [8]:
mol_base = qtk.Molecule('xyz/ge.xyz')
# mol_base.extend([2,2,2])
# mol_base.sort(order='xyz')

# ANM space and finite difference matrix
ANM space is spanned by the PP parameters. For FCC primitive cell of two atoms, there are 30 parameters (15 per atoms).

The goal is the construct matrix elements 
$$
\begin{array}{rcl}
\mathbf{H}_{ij}&=&\frac{\partial^2 E}{\partial\sigma_i\partial\sigma_j}\\
&=&\frac{\partial}{\partial\sigma_i}\Big(\frac{\partial E}{\partial\sigma_j}\Big)\\
&\approx&
\frac{\partial}{\partial\sigma_i}\Big(\frac{E(\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i, \sigma_j)}{\Delta\sigma_j}\Big)\\
&\approx&
\frac{1}{\Delta\sigma_i}\Bigg(\frac{E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i+\Delta\sigma_i, \sigma_j)}{\Delta\sigma_j}
- \frac{E(\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i, \sigma_j)}{\Delta\sigma_j}\Bigg).
\end{array}
$$

That is, there are four finite difference calculations required for each of the matrix elements:
$E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j)$, 
$E(\sigma_i+\Delta\sigma_i, \sigma_j)$, 
$E(\sigma_i, \sigma_j+\Delta\sigma_j)$, 
$E(\sigma_i, \sigma_j)$, 
where only the first term is unique for each element.

Note that the finite difference formula is different for diagonal terms $\mathbf{H}_{ii} = \frac{E(\sigma_i+\Delta\sigma_i) - 2E(\sigma_i) + E(\sigma_i-\Delta\sigma_i)}{\Delta\sigma_i^2}$.

The required finite difference calculations are
* $E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j)$: $N(N-1)/2$ calculations for $i\neq j$
* $E(\sigma_i+\Delta\sigma_i)$: $N$ calculations
* $E(\sigma_i-\Delta\sigma_i)$: $N$ calculations
* $E(\sigma_i, \sigma_j)$: 1 calculation

which added up to $\frac{N^2}{2}+\frac{3}{2}N+1$ calculations, where $N$ is the number of parameters in the system.

# Function interface
The following functions are required for reusable/extensive code

## _crystal to parameter space map_: cyl2par
input: crystal object

output: parameter space coordinate

## _parameter spsce to crystal map_: par2cyl
input: parameter space coordinate

output: crystal object

## _finite difference crystal construction_: FDcyl
input: finite difference indices, step size

output: QMInp calculation object

__NOTE__: for the construction of FD matrix, the index information will be written on the file name for lazy processing

## _finite difference matrix construction_: H_FD
input: finite difference calculation folder path __with fomated index file name__

output: finite difference matrix as 2D numpy array

### TODO:
implement CPMD interface to take PP object as input and write PP file on the fly __DONE__

In [11]:
def cyl2par(mol, size=[1, 3]):
    par = []
    for a in mol.type_list:
        crd = qtk.PP(a, size=size)
        vec = crd.vectorize()
        par.append([vec[0][1:], a, vec[0][0]])
    return par

def par2cyl(par, mol_base=mol_base, size=[1, 3]):
    mol = mol_base.copy()
    mol.sort(order='xyz')
    for i, crd in enumerate(par):
        pp = qtk.PP(crd[1], size=size)
        vec = [crd[2]]
        vec.extend(crd[0])
        pp.unvectorize(vec, size[0], range(size[1], 0, -1))
        pp.name = crd[1] + '%02d' % i
        mol.setAtoms(i, string=pp)
    return mol

mol = par2cyl(cyl2par(mol_base))
inp = qtk.QMInp(mol, **qmsetting)
inp.write()

&INFO
 ge
&END

&CPMD
 MIRROR
 BENCHMARK
  1 0 0 0 0 0 0 0 0 0
 OPTIMIZE WAVEFUNCTION
 CONVERGENCE ORBITAL
  1.00E-05
 MAXITER
  1000
 CENTER MOLECULE OFF
 MEMORY BIG
&END

&DFT
 FUNCTIONAL PBE
&END

&SYSTEM
 CELL VECTORS
     0.000000    2.715500    2.715500
     2.715500    0.000000    2.715500
     2.715500    2.715500    0.000000
 ANGSTROM
 CUTOFF
  100.0
 KPOINTS MONKHORST-PACK
  1 1 1
&END

&ATOMS
*Ge01.psp
 LMAX=F
   1
     3.8411   3.8411   3.8411
*Ge00.psp
 LMAX=F
   1
     0.0000   0.0000   0.0000
&END


In [13]:
def FDcyl(mol, i, j, step, size=[1,3], dim=15):
    """
    input:  molecule object: where the FD perturbation will be introduced
            i, j: integer indices for the perturbation matrix row and column number
            step: absolute step size for finite difference perturbation
            size: PP configuration space
    output: molecule object with appropreate PP and file names attached
    """
    def get_pp(I, ip, font, mode='+'):
        if type(mol.string[I]) is type(qtk.PP()):
            pp = mol.string[I]
        else:
            pp = qtk.PP(mol.type_list[I], size=size)
            pp.name = mol.type_list[I]
        vec = pp.vectorize()[0]
        new_pp = pp.copy()
        if mode == '+':
            vec[1+ip] += step
        elif mode == '-':
            vec[1+ip] -= step
        new_pp.unvectorize(vec, size[0], range(size[1], 0, -1))
        new_pp.name += '_%s%03dp%03d' % (font, I, ip)
        return new_pp
    
    # first order
    if i == j:
        I, ip = divmod(i, dim)
        modes = [['p', '+'], ['m', '-']]
        mols = []
        for m in modes:
            mol_new = mol.copy()
            pp = get_pp(I, ip, 'I', mode=m[1])
            mol_new.setAtoms(I, string=pp)
            mol_new.name += '_i%03d%s-I%02dp%02d' % (i, m[0], I, ip)
            mols.append(mol_new)
        return mols
    # second order
    else:
        mol = mol.copy()
        inds = [i, j]
        ind_font = ['I', 'J']
        for t, s in enumerate([i, j]):
            I, ip = divmod(s, dim)
            inds.append(I)
            inds.append(ip)
            pp = get_pp(I, ip, ind_font[t])
            mol.setAtoms(I, string=pp)
        mol.name += "_i%03dj%03d-I%02dp%02d-J%02dp%02d" % tuple(inds)
    return [mol]

# mol_test = FDcyl(mol_base, 5, 28, 100)[0]
# inp = qtk.QMInp(mol_test, **qmsetting)
# inp.write('PPTest', overwrite=True)

## Construct FD crystal for calculations

In [ ]:
mols_fd = []
mols_ref = []
par = cyl2par(mol_base)

# get dimension of PP space
# single atom PP space * number of atoms
N = len(par[0][0]) * len(par)

for info_cyl in cyl_map:
    mol = mol_base.copy()
    for i in range(mol.N):
        mol.setAtoms(i, Z=info_cyl[2][i])
        mol.name = info_cyl[0]
        for info_a in cyl_map:
            if info_a != info_cyl:
                mol_a = mol.copy()
                mol_a.name += '_a-' + info_a[0]
                mol_a.setCelldm([info_a[1], info_a[1], info_a[1], 0, 0, 0])
                mols_fd.append(mol_a)
                mols_ref.append(mol_a)
                # get off-diagonal terms
                for i in range(N):
                    for j in range(i, N):
                        mols = FDcyl(mol_a, i, j, step=0.01)
                        mols_fd.extend(mols)
        
inps_fd = [qtk.QMInp(mol_base, **qmsetting)]
for m in mols_fd:
    inps_fd.append(qtk.QMInp(m, **qmsetting))

In [9]:
mols_ref

[ge_a-si,
 ge_a-ge,
 ge_a-sige,
 ge_a-sn,
 ge_a-snsi,
 ge_a-gesn,
 ge_a-gaas,
 ge_a-alp,
 ge_a-alas,
 ge_a-alsb,
 ge_a-gap,
 ge_a-gasb,
 ge_a-inp,
 ge_a-inas,
 ge_a-insb]

In [10]:
inps_ref = []
for mol in mols_ref:
    inp = qtk.QMInp(mol, **qmsetting)
    inps_ref.append(inp)

In [12]:
qtk.qmRunAll(inps_ref, "ge_ref")

qmRunJob: runing qmjob:'ge_a-si: cpmd' with name: ge_a-si 
qmRunJob: runing qmjob:'ge_a-sige: cpmd' with name: ge_a-sige 
QMInp.run: running job with command: cpmd.x ge_a-si.inp
QMInp.run: running job with command: cpmd.x ge_a-sige.inp

qmRunJob: runing qmjob:'ge_a-sn: cpmd' with name: ge_a-sn 
qmRunJob: runing qmjob:'ge_a-ge: cpmd' with name: ge_a-ge 
QMInp.run: running job with command: cpmd.x ge_a-sn.inp

QMInp.run: running job with command: cpmd.x ge_a-ge.inp


qmRunJob: runing qmjob:'ge_a-snsi: cpmd' with name: ge_a-snsi 
QMInp.run: running job with command: cpmd.x ge_a-snsi.inp

qmRunJob: runing qmjob:'ge_a-gesn: cpmd' with name: ge_a-gesn 
QMInp.run: running job with command: cpmd.x ge_a-gesn.inp

qmRunJob: runing qmjob:'ge_a-gaas: cpmd' with name: ge_a-gaas 
QMInp.run: running job with command: cpmd.x ge_a-gaas.inp

qmRunJob: runing qmjob:'ge_a-alp: cpmd' with name: ge_a-alp 
QMInp.run: running job with command: cpmd.x ge_a-alp.inp

qmRunJob: runing qmjob:'ge_a-alas: cpmd' with

In [78]:
inp = qtk.QMInp(mol_base, debug=True, **qmsetting)
inp.run('ge', overwrite=True, debug=True)

Overwrite existing folder ge
QMInp.run: running job with command: cpmd.x ge.inp

ge.out failed with message: list index out of range


nan

In [73]:
mol_base.string

['', '']

In [70]:
qtk.pdump(inps_fd, 'ge_H_scan_inps')

In [71]:
qtk.setting.quiet = True
#qtk.qmWriteAll(inps_fd, 'ge_H_scan')
qtk.setting.quiet = False

## Test of PP interface

In [68]:
# pp = qtk.PP('Ge', size=[1,3])
# print pp
# print pp.param
# v = pp.vectorize()[0]
# for i in range(2, 7):
#     v[i] = 10
# new = pp.copy().unvectorize(v, 1, [3,2,1])

# print pp.param
# print new.param